#**Imports**

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import pickle
from keras.models import load_model
from pickle import load

#**Data preprocessing**

This code performs data loading and preprocessing for a text file containing names. It divides the names into sequences of length 3 and creates a mapping of unique characters to integers. By changing the input data file, you can generate a corresponding model for the new dataset by rerunning the code cells.

In [2]:
# load data
def load_doc(filename):
    with open(filename, 'r', encoding="utf8") as file:
        text = file.read()
    return text

# tokenization, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    with open(filename, 'w', encoding="utf8") as file:
        file.write(data)


raw_text = load_doc('names.txt')
print(raw_text)

# start preprocess
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# devide into seq
seq_len = 3
step = 1
sequences = []
next_chars = []
for i in range(0, len(raw_text) - seq_len, step):
    sequences.append(raw_text[i:i + seq_len])
    next_chars.append(raw_text[i + seq_len])
print('Total Sequences:', len(sequences))

# create a mapping of unique characters to integers , in other words dict with an int for each charcter
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# create input and output pairs encoded as integers
X = np.zeros((len(sequences), seq_len), dtype=np.int32)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, seq in enumerate(sequences):
    for j, char in enumerate(seq):
        X[i, j] = char_to_int[char]
    y[i, char_to_int[next_chars[i]]] = 1

﻿آلاء آمال آمنة آيات
آية أبرار أثيل أحلام
أروى أريج  أزهار أسارير
أسماء أسمى أسيل أشجان
أطياف أغاريد أفراح أفكار
أفنان ألطاف ألوف أم كلثوم
أمارة أمال أمامة أماميان
أمانة أماني أمرات أملة
أمنية أمورة أمونة أمية
أميرة أميمة أميمية أمينة
أمينة أناة أنار أنارات
أنسام أنسة أنسوقة أنشودة
أنصار أنظارأنظام أنغم
أنوار أنوس أنوف أنيسة 
أنيسة أهداف أوبية أودة
أوسال أوصاف أوصال أولياء
أوها إباء إجلالإحسان
إخلاص إدراك إسراء إسعاد
إسعاف إشراف إصلاح إعزاز 
إقبال إقدام إكليل إمداد
إمهال إناس إنجاح إنشاد
إنصاف إنعام إهداف إهلال 
إيتاء إيثار إيحاء إيداع
إيضاح إيفاء إيماء إيمان
إيناس ابتسام ابتهاج ابتهال
اجتهاد احتشام ازدهاراعتدال
اعتماد افتخار افتكار الحد
الحدة الدرداء السعدية السكينة
النعيمة امتثال امتيازانبساط 
انبهاج انتصارانتظار انتهاء
انجرار انسجام انشراح انشراف 
انشقار انفصال بائقة بادرة 
بارزة بارعة باركة بارية
بازعة باسة باسطة باسقة
باسلة بتول بثينة بحرية
براء بسمة بسيمة بشرى 
بشيرة بلقيس بليغة بهيجة
بهيسة بيبي تانيا تغاني
تمارة تماضر تنال تودد
تيسير تيماء تيمة ثراء 
ثوابة ثوبة ثويبة جبرة 
جبلة 

<ipython-input-2-d74f71ce9e37>:38: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


# **Modelization**

In [3]:
# define the LSTM model
model = Sequential()
model.add(Embedding(len(chars), 128, input_length=seq_len))
model.add(LSTM(256))
model.add(Dense(len(chars), activation='softmax'))
print(model.summary())

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# define early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=[early_stop])

# save the model to file
model.save('model.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 128)            4864      
                                                                 
 lstm (LSTM)                 (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 38)                9766      
                                                                 
Total params: 408,870
Trainable params: 408,870
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
21/21 [==============================] - 6s 59ms/step - loss: 3.5061 - accuracy: 0.1708
Epoch 2/50
21/21 [==============================] - 2s 80ms/step - loss: 2.9712 - accuracy: 0.1966
Epoch 3/50
21/21 [==============================] - 2s 95ms/step - loss: 2.7583 - accuracy: 0.2424
Epoch 4/50
2

#**Prediction**

In [4]:
def generate_seq(model,  seed_text, num_chars):
    seq_len=len(seed_text)
    generated = seed_text
    for i in range(num_chars):
        # encode the characters as integers
        x_pred = np.zeros((1, seq_len), dtype=np.int32)
        for j, char in enumerate(seed_text):
            x_pred[0, j] = char_to_int[char]
        # predict character
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = int_to_char[next_index]
        # append to input
        generated += next_char
        seed_text = seed_text[1:] + next_char
    return generated

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))
print(generate_seq(model, 'أوس', 2)) # should print 'أوسام'
print(generate_seq(model, 'نبي', 3)) # should print 'نبيهة'
print(generate_seq(model, 'مثا', 2)) # should print 'مثابة'



أوسام
نبيهة 
مثابة


# **Full code**

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import pickle
from keras.models import load_model
from pickle import load

# load data
def load_doc(filename):
    with open(filename, 'r', encoding="utf8") as file:
        text = file.read()
    return text

# tokenization, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    with open(filename, 'w', encoding="utf8") as file:
        file.write(data)


raw_text = load_doc('names.txt')
print(raw_text)

# start preprocess
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# devide into seq
seq_len = 3
step = 1
sequences = []
next_chars = []
for i in range(0, len(raw_text) - seq_len, step):
    sequences.append(raw_text[i:i + seq_len])
    next_chars.append(raw_text[i + seq_len])
print('Total Sequences:', len(sequences))

# create a mapping of unique characters to integers , in other words dict with an int for each charcter
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# create input and output pairs encoded as integers
X = np.zeros((len(sequences), seq_len), dtype=np.int32)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, seq in enumerate(sequences):
    for j, char in enumerate(seq):
        X[i, j] = char_to_int[char]
    y[i, char_to_int[next_chars[i]]] = 1

# define the LSTM model
model = Sequential()
model.add(Embedding(len(chars), 128, input_length=seq_len))
model.add(LSTM(256))
model.add(Dense(len(chars), activation='softmax'))
print(model.summary())

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# define early stopping callback
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=[early_stop])

# save the model to file
model.save('model.h5')
# save the mapping
with open('mapping.pkl', 'wb') as file:
    pickle.dump(char_to_int, file)
    pickle.dump(int_to_char, file)

# generate a sequence of characters with a trained model
def generate_seq(model,  seed_text, num_chars):
    seq_len=len(seed_text)
    generated = seed_text
    for i in range(num_chars):
        # encode the characters as integers
        x_pred = np.zeros((1, seq_len), dtype=np.int32)
        for j, char in enumerate(seed_text):
            x_pred[0, j] = char_to_int[char]
        # predict character
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = int_to_char[next_index]
        # append to input
        generated += next_char
        seed_text = seed_text[1:] + next_char
    return generated

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

print(generate_seq(model, 'أوس', 2)) # should print 'أوسام'
print(generate_seq(model, 'نبي', 3)) # should print 'نبيهة'
print(generate_seq(model, 'مثا', 2)) # should print 'مثابة'




﻿آلاء آمال آمنة آيات
آية أبرار أثيل أحلام
أروى أريج  أزهار أسارير
أسماء أسمى أسيل أشجان
أطياف أغاريد أفراح أفكار
أفنان ألطاف ألوف أم كلثوم
أمارة أمال أمامة أماميان
أمانة أماني أمرات أملة
أمنية أمورة أمونة أمية
أميرة أميمة أميمية أمينة
أمينة أناة أنار أنارات
أنسام أنسة أنسوقة أنشودة
أنصار أنظارأنظام أنغم
أنوار أنوس أنوف أنيسة 
أنيسة أهداف أوبية أودة
أوسال أوصاف أوصال أولياء
أوها إباء إجلالإحسان
إخلاص إدراك إسراء إسعاد
إسعاف إشراف إصلاح إعزاز 
إقبال إقدام إكليل إمداد
إمهال إناس إنجاح إنشاد
إنصاف إنعام إهداف إهلال 
إيتاء إيثار إيحاء إيداع
إيضاح إيفاء إيماء إيمان
إيناس ابتسام ابتهاج ابتهال
اجتهاد احتشام ازدهاراعتدال
اعتماد افتخار افتكار الحد
الحدة الدرداء السعدية السكينة
النعيمة امتثال امتيازانبساط 
انبهاج انتصارانتظار انتهاء
انجرار انسجام انشراح انشراف 
انشقار انفصال بائقة بادرة 
بارزة بارعة باركة بارية
بازعة باسة باسطة باسقة
باسلة بتول بثينة بحرية
براء بسمة بسيمة بشرى 
بشيرة بلقيس بليغة بهيجة
بهيسة بيبي تانيا تغاني
تمارة تماضر تنال تودد
تيسير تيماء تيمة ثراء 
ثوابة ثوبة ثويبة جبرة 
جبلة 

<ipython-input-5-40bc9d5c91c4>:47: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 128)            4864      
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dense_1 (Dense)             (None, 38)                9766      
                                                                 
Total params: 408,870
Trainable params: 408,870
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
21/21 [==============================] - 3s 35ms/step - loss: 3.4983 - accuracy: 0.1682
Epoch 2/50
21/21 [==============================] - 1s 34ms/step - loss: 2.9863 - accuracy: 0.1966
Epoch 3/50
21/21 [==============================] - 1s 34ms/step - loss: 2.8010 - accuracy: 0.2406
Epoch 4/50